## **Notebook Overview**

### **0.2.1 Project Overview**
This notebook builds upon the preprocessed data from the previous notebook (`01_data_loading_and_preprocessing.ipynb`) and focuses on feature selection, data preparation, and outlier handling. These steps involve refining the dataset by selecting the most relevant features, handling outliers by flagging them, and preparing the data for machine learning models.

### **0.2.2 Why is This Important?**
Effective feature selection and data preparation are critical to the success of machine learning projects. By reducing the dataset's complexity and addressing data quality issues such as missing values, outliers, and duplicates, we can improve model performance, interpretability, and generalization. Handling outliers is particularly important as they can skew predictions, and flagging them preserves potentially valuable information without directly altering the dataset.

### **0.2.3 Dataset Description**
We are continuing to work with the Home Credit dataset, which contains loan application data. This dataset includes a variety of features related to applicant demographics, financial history, and loan specifics. The data has already been cleaned and memory-optimized in the previous notebook. Now, we focus on feature reduction, outlier handling, and imputation of missing values.

### **0.2.4 Our Approach**
In this notebook, we will employ several techniques for feature selection, outlier handling, and data preparation:

1. **Missing Value Imputation:** We will use KNN imputation for numerical features and mode imputation for categorical features to handle missing data.
2. **Outlier Detection and Flagging:** We will identify outliers using statistical methods such as IQR or Z-scores and flag them by adding a binary feature to indicate the presence of outliers.
3. **Feature Reduction:** We will remove features with high percentages of missing values, low variance, or low correlation with the target variable, while ensuring essential features are retained.
4. **Duplicate Removal:** We will identify and remove duplicate rows to maintain data integrity.
5. **Save Preprocessed Data:** Save the prepared datasets in Parquet format for use in subsequent notebooks (EDA and modeling).

### **0.2.5 Additional Notes**
This notebook assumes that the preprocessed data from Notebook 1 is available in the `../data/processed/` directory. The refined data, with outliers flagged and irrelevant features removed, will be saved in the same directory for further exploration and modeling.

In [1]:
import polars as pl

from retail_bank_risk.data_preprocessing_utils import (
    initial_feature_reduction,
    impute_numerical_features,
    impute_categorical_features,
    count_duplicated_rows,
    analyze_missing_values,
    detect_anomalies_iqr,
    flag_anomalies,
)

In [3]:
application_train = pl.read_parquet(
    "../data/processed/application_train_preprocessed.parquet"
)
application_test = pl.read_parquet(
    "../data/processed/application_test_preprocessed.parquet"
)

**To improve our training and testing datasets, we will remove features based on missing values, variance, and correlation. Features with a missing value ratio above `missing_threshold` (50%) or a variance below `variance_threshold` will be removed.**

Columns with a variance less than or equal to 0.01 are considered to have insufficient variability and are removed from the dataset. This is because a variance of 0.01 implies very little change in the values of that feature, making it potentially less useful for distinguishing between different observations in predictive modeling tasks.

Additionally, we'll calculate the correlation between numerical features and the target variable (`target`). **We recognize the importance of income in credit risk assessment, so the `amt_income_total` feature will always be retained regardless of its correlation with the target.** Other features that show a low correlation with the target (absolute correlation value below the specified `correlation_threshold` - default 0.05) will be removed.

**This entire feature reduction process will be applied separately to the training and testing datasets. While the criteria (missing value, variance, and correlation thresholds) are determined from the training data, we'll use the same set of selected features to reduce both the training and test datasets.** This ensures consistency in our feature sets and prevents data leakage from the test set into the feature selection process.

This process will improve data quality, reduce dimensionality, and improve the performance of the credit risk model.


In [4]:
essential_features = [
    "amt_income_total",
    "amt_credit",
    "amt_annuity",
    "amt_goods_price",
]
target_column = "target"

reduced_train, reduced_test = initial_feature_reduction(
    application_train,
    application_test,
    target_column,
    essential_features=essential_features,
)

print(f"Original number of features in train: {application_train.shape[1]}")
print(f"Number of features in reduced train: {reduced_train.shape[1]}")
print(f"Original number of features in test: {application_test.shape[1]}")
print(f"Number of features in reduced test: {reduced_test.shape[1]}")

sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


Original number of features in train: 122
Number of features in reduced train: 27
Original number of features in test: 121
Number of features in reduced test: 26


In [5]:
reduced_train.head()

target,days_last_phone_change,ext_source_2,reg_city_not_work_city,region_rating_client_w_city,ext_source_3,days_id_publish,days_birth,region_rating_client,name_contract_type,code_gender,flag_own_car,flag_own_realty,name_type_suite,name_income_type,name_education_type,name_family_status,name_housing_type,occupation_type,weekday_appr_process_start,organization_type,housetype_mode,emergencystate_mode,amt_income_total,amt_credit,amt_annuity,amt_goods_price
i8,f32,f32,i8,i8,f32,i32,i32,i8,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32
1,-1134.0,0.262949,0,2,0.139376,-2120,-9461,2,"""Cash loans""","""M""","""N""","""Y""","""Unaccompanied""","""Working""","""Secondary / secondary special""","""Single / not married""","""House / apartment""","""Laborers""","""WEDNESDAY""","""Business Entity Type 3""","""block of flats""","""No""",202500.0,406597.5,24700.5,351000.0
0,-828.0,0.622246,0,1,null,-291,-16765,1,"""Cash loans""","""F""","""N""","""N""","""Family""","""State servant""","""Higher education""","""Married""","""House / apartment""","""Core staff""","""MONDAY""","""School""","""block of flats""","""No""",270000.0,1293502.5,35698.5,1.1295e6
0,-815.0,0.555912,0,2,0.729567,-2531,-19046,2,"""Revolving loans""","""M""","""Y""","""Y""","""Unaccompanied""","""Working""","""Secondary / secondary special""","""Single / not married""","""House / apartment""","""Laborers""","""MONDAY""","""Government""",null,null,67500.0,135000.0,6750.0,135000.0
0,-617.0,0.650442,0,2,null,-2437,-19005,2,"""Cash loans""","""F""","""N""","""Y""","""Unaccompanied""","""Working""","""Secondary / secondary special""","""Civil marriage""","""House / apartment""","""Laborers""","""WEDNESDAY""","""Business Entity Type 3""",null,null,135000.0,312682.5,29686.5,297000.0
0,-1106.0,0.322738,1,2,null,-3458,-19932,2,"""Cash loans""","""M""","""N""","""Y""","""Unaccompanied""","""Working""","""Secondary / secondary special""","""Single / not married""","""House / apartment""","""Core staff""","""THURSDAY""","""Religion""",null,null,121500.0,513000.0,21865.5,513000.0


In [6]:
categorical_cols_train = [
    "region_rating_client",
    "region_rating_client_w_city",
    "reg_city_not_work_city",
    "target",
]

categorical_cols_test = [
    "region_rating_client",
    "region_rating_client_w_city",
    "reg_city_not_work_city",
]

reduced_train = reduced_train.with_columns(
    [
        pl.col(col).round(0).cast(pl.Int32).cast(pl.Utf8).cast(pl.Categorical)
        for col in categorical_cols_train
    ]
)

reduced_test = reduced_test.with_columns(
    [
        pl.col(col).round(0).cast(pl.Int32).cast(pl.Utf8).cast(pl.Categorical)
        for col in categorical_cols_test
    ]
)

As we have adjusted the categorical features, next, we'll identify and handle missing values in the remaining features. Afterward, we'll analyze feature importance to reduce noise and improve model performance.

Given the high number of features (84), we'll analyze the top 5 features in both the training and test datasets to understand their characteristics and determine the most suitable imputation method.


In [7]:
analyze_missing_values(reduced_train, reduced_test)

Top 5 columns with missing values in reduced train set:
shape: (5, 2)
┌─────────────────────┬────────────────────┐
│ column              ┆ missing_percentage │
│ ---                 ┆ ---                │
│ str                 ┆ f64                │
╞═════════════════════╪════════════════════╡
│ housetype_mode      ┆ 50.18              │
│ emergencystate_mode ┆ 47.4               │
│ occupation_type     ┆ 31.35              │
│ ext_source_3        ┆ 19.83              │
│ name_type_suite     ┆ 0.42               │
└─────────────────────┴────────────────────┘

Top 5 columns with missing values in reduced test set:
shape: (5, 2)
┌─────────────────────┬────────────────────┐
│ column              ┆ missing_percentage │
│ ---                 ┆ ---                │
│ str                 ┆ f64                │
╞═════════════════════╪════════════════════╡
│ housetype_mode      ┆ 48.46              │
│ emergencystate_mode ┆ 45.56              │
│ occupation_type     ┆ 32.01              │
│ ext

We can see that the credit risk dataset contains some missing values.

Because outliers might be significant in this context, reflecting genuine differences in individual risk levels, outlier treatment will not be performed before KNN imputation for numerical features.


In [8]:
reduced_train, reduced_test = impute_numerical_features(
    reduced_train, reduced_test, target_column
)

Next up, for categorical features with high missing value percentages, we will utilize mode imputation.

This approach, while potentially introducing bias, is deemed suitable due to its simplicity, minimal computational cost, and limited impact on overall dataset representation only few features.

In case of bad results, we could just drop them.


In [9]:
reduced_train, reduced_test = impute_categorical_features(
    reduced_train, reduced_test, target_column
)

/Users/vytautasbunevicius/retail-bank-risk-evaluation/src/retail_bank_risk/data_preprocessing_utils.py:268: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  train_df = train_df.with_columns(pl.col(col).fill_null("mode"))
/Users/vytautasbunevicius/retail-bank-risk-evaluation/src/retail_bank_risk/data_preprocessing_utils.py:269: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  test_df = test_df.with_columns(pl.col(col).fill_null("mode"))


Lastly, we confirm if all of the missing values were addressed.


In [10]:
analyze_missing_values(reduced_train, reduced_test)

No missing values found in the reduced train set.
No missing values found in the reduced test set.


Next, we'll address potential duplicate entries in our training dataset.

Although we removed the unique client identifier (`sk_id_curr`) during feature selection, we can still check for duplicates based on a combination of features that should be unique or nearly unique for a single loan application.

We'll focus on exact matches across these features, as even minor variations might represent distinct applications.

If we find any duplicates, we'll remove them to ensure the integrity and quality of our training data.


In [11]:
count_duplicated_rows(reduced_train)

The DataFrame contains 0 duplicated rows.


As we have found no duplicate values, there were no unique id, so we checked on our selected columns, and were seem to not find any of the duplicate values. Next up we will check for outliers, and flag them without removing, as the credit data is sensitive and may indicate sensitive information.

In [12]:
pd_application_train = reduced_train.to_pandas()
pd_application_test = reduced_test.to_pandas()

In [14]:
numerical_features_financial = [
    "ext_source_2",
    "ext_source_3",
    "amt_income_total",
    "amt_credit",
]

numerical_features_loan_specifics = ["amt_annuity", "amt_goods_price"]

numerical_features_timing_history = [
    "days_last_phone_change",
    "days_birth",
    "days_id_publish",
]

all_numerical_features = (
    numerical_features_timing_history
    + numerical_features_financial
    + numerical_features_loan_specifics
)

In [15]:
anomalies = detect_anomalies_iqr(pd_application_train, all_numerical_features)
print("Number of anomalies detected:", len(anomalies))

Anomalies detected in feature 'days_last_phone_change':
       reg_city_not_work_city region_rating_client_w_city  \
2847                        0                           2   
2972                        0                           1   
4533                        1                           3   
4660                        0                           2   
5599                        0                           1   
...                       ...                         ...   
304373                      0                           1   
304395                      0                           2   
304873                      0                           1   
304878                      0                           2   
306923                      1                           2   

       region_rating_client name_contract_type code_gender flag_own_car  \
2847                      2         Cash loans           M            N   
2972                      1    Revolving loans           F   

The results show specific loan applications flagged as anomalies in features like phone change frequency, external data sources, income, credit amount, and goods price. A total of 29,697 anomalies were identified across these features.

We'll create a new feature, `is_anomaly`, to flag these anomalous applications. It will be set to 1 for rows with anomalies in any of the identified features and 0 otherwise. This flag can be used for further analysis and modeling considerations. 


In [16]:
pd_application_train["is_anomaly"] = flag_anomalies(
    pd_application_train, all_numerical_features
)
pd_application_test["is_anomaly"] = flag_anomalies(
    pd_application_test, all_numerical_features
)

As we have flagged our outliers, we will move on to the next notebook, where we will perform our EDA. We will save our Pandas DataFrames as Parquet files.

In [18]:
pd_application_train.to_parquet("../data/processed/application_train_prepared.parquet")
pd_application_test.to_parquet("../data/processed/application_test_prepared.parquet")